In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, BatchNormalization, Input, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import ReLU
from tensorflow.keras.models import Model

In [7]:
def inception_naive_module(x, f1, f2, f3):
    c1 = Conv2D(f1, (1, 1), padding='same', activation='relu')(x)
    c2 = Conv2D(f2, (3, 3), padding='same', activation='relu')(x)
    c3 = Conv2D(f3, (5, 5), padding='same', activation='relu')(x)
    pool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    out = concatenate(c1, c2, c3, pool)
    return out

In [8]:
def conv2d_bn(x, filters, filter_size, padding='same', strides=(1, 1)):
    x = Conv2D(filters, filter_size, padding=padding, strides=strides)(x)
    x = BatchNormalization(axis=3, scale=False)(x)
    x = ReLU()(x)
    return x

In [17]:
def inc_block_a(x):
    first_branch = conv2d_bn(x, 64, (1, 1),  padding='same', strides=(1, 1))

    second_branch = conv2d_bn(x, 48, (1, 1),  padding='same', strides=(1, 1))
    second_branch = conv2d_bn(second_branch, 64, (5, 5),  padding='same', strides=(1, 1))

    third_branch = conv2d_bn(x, 64, (1, 1),  padding='same', strides=(1,1))
    third_branch = conv2d_bn(third_branch, 96, (3, 3),  padding='same', strides=(1, 1))
    third_branch = conv2d_bn(third_branch, 96, (3, 3),  padding='same', strides=(1, 1))

    fourth_branch = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    fourth_branch = conv2d_bn(fourth_branch, 32, (1, 1),  padding='same', strides=(1, 1))

    x = concatenate([first_branch, second_branch, third_branch, fourth_branch], axis=3)
    return x

In [18]:
def reduction_block_a(x):
    first_branch = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(x)

    second_branch = conv2d_bn(x, 384, (3, 3), padding='valid', strides=(2, 2))

    third_branch = conv2d_bn(x, 64, (1, 1), padding='same', strides=(1, 1))
    third_branch = conv2d_bn(third_branch, 96, (3, 3), padding='same', strides=(1, 1))
    third_branch = conv2d_bn(third_branch, 96, (3, 3), padding='valid', strides=(2, 2))

    x = concatenate([first_branch, second_branch, third_branch], axis=3)
    return x


In [59]:
def inc_block_b(x):
    first_branch = conv2d_bn(x, 192, (1, 1),  padding='same', strides=(1, 1))

    second_branch = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    second_branch = conv2d_bn(second_branch, 192, (1, 1),  padding='same', strides=(1, 1))

    third_branch = conv2d_bn(x, 128, (1, 1),  padding='same', strides=(1,1))
    third_branch = conv2d_bn(third_branch, 128, (1, 7),  padding='same', strides=(1, 1))
    third_branch = conv2d_bn(third_branch, 192, (7, 1),  padding='same', strides=(1, 1))

    fourth_branch = conv2d_bn(x, 128, (1, 1),  padding='same', strides=(1,1))
    fourth_branch = conv2d_bn(fourth_branch, 128, (7, 1),  padding='same', strides=(1, 1))
    fourth_branch = conv2d_bn(fourth_branch, 128, (1, 7),  padding='same', strides=(1, 1))
    fourth_branch = conv2d_bn(fourth_branch, 128, (7, 1),  padding='same', strides=(1, 1))
    fourth_branch = conv2d_bn(fourth_branch, 192, (1, 7),  padding='same', strides=(1, 1))

    x = concatenate([first_branch, second_branch, third_branch, fourth_branch], axis=3)
    return x

In [48]:
def reduction_block_b(x):
    first_branch = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(x)

    second_branch = conv2d_bn(x, 192, (1, 1),  padding='same', strides=(1, 1))
    second_branch = conv2d_bn(second_branch, 320, (3, 3),  padding='valid', strides=(2, 2))

    third_branch = conv2d_bn(x, 192, (1, 1),  padding='same', strides=(1, 1))
    third_branch = conv2d_bn(third_branch, 192, (1, 7),  padding='same', strides=(1, 1))
    third_branch = conv2d_bn(third_branch, 192, (7, 1),  padding='same', strides=(1, 1))
    third_branch = conv2d_bn(third_branch, 192, (3, 3),  padding='valid', strides=(2, 2))

    x = concatenate([first_branch, second_branch, third_branch], axis=3)
    return x

In [58]:
def inc_block_c(x):
    first_branch = conv2d_bn(x, 320, (1, 1),  padding='same', strides=(1, 1))

    second_branch = conv2d_bn(x, 384, (1, 1),  padding='same', strides=(1, 1))
    second_branch_1 = conv2d_bn(second_branch, 384, (1, 3),  padding='same', strides=(1, 1))
    second_branch_2 = conv2d_bn(second_branch, 384, (3, 1),  padding='same', strides=(1, 1))
    second_branch = concatenate([second_branch_1, second_branch_2], axis=3)

    third_branch = conv2d_bn(x, 448, (1, 1),  padding='same', strides=(1,1))
    third_branch = conv2d_bn(third_branch, 384, (3, 3),  padding='same', strides=(1, 1))
    third_branch_1 = conv2d_bn(third_branch, 384, (1, 3),  padding='same', strides=(1, 1))
    third_branch_2 = conv2d_bn(third_branch, 384, (3, 1),  padding='same', strides=(1, 1))
    third_branch = concatenate([third_branch_1, third_branch_2], axis=3)

    fourth_branch = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    fourth_branch = conv2d_bn(fourth_branch, 192, (1, 1),  padding='same', strides=(1, 1))

    x = concatenate([first_branch, second_branch, third_branch, fourth_branch], axis=3)
    return x

In [70]:
def Inception_V3():
    input = Input(shape=(299, 299, 3))

    x = conv2d_bn(input, 32, (3, 3),  padding='valid', strides=(2,2))
    x = conv2d_bn(x, 32, (3, 3),  padding='valid', strides=(1,1))
    x = conv2d_bn(x, 64, (3, 3),  padding='same', strides=(1,1))

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(x)

    x = conv2d_bn(x, 80, (1, 1),  padding='valid', strides=(1,1))
    x = conv2d_bn(x, 192, (3, 3),  padding='valid', strides=(1,1))

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(x)

    x = inc_block_a(x)
    x = inc_block_a(x)
    x = inc_block_a(x)

    x = reduction_block_a(x)

    x = inc_block_b(x)
    x = inc_block_b(x)
    x = inc_block_b(x)
    x = inc_block_b(x)

    x = reduction_block_b(x)

    x = inc_block_c(x)
    x = inc_block_c(x)

    x = GlobalAveragePooling2D(name='avg_pool')(x)

    x = Dense(2048, activation='relu')(x)

    x = Dense(1000, activation='softmax', name='predictions')(x)

    model = Model(input, x, name='InceptionV3')

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [71]:
model = Inception_V3()

In [72]:
model.summary()

Model: "InceptionV3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_18            │ (None, 299, 299, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1344 (Conv2D)      │ (None, 149, 149, 32)   │            896 │ input_layer_18[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1344  │ (None, 149, 149, 32)   │             96 │ conv2d_1344[0][0]      │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_1344 (ReLU)         │ (None, 149, 149, 32)   │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1345 (Conv2D)      │ (None, 147, 147, 32)   │          9,248 │ re_lu_1344[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1345  │ (None, 147, 147, 32)   │             96 │ conv2d_1345[0][0]      │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_1345 (ReLU)         │ (None, 147, 147, 32)   │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1346 (Conv2D)      │ (None, 147, 147, 64)   │         18,496 │ re_lu_1345[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1346  │ (None, 147, 147, 64)   │            192 │ conv2d_1346[0][0]      │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_1346 (ReLU)         │ (None, 147, 147, 64)   │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_122         │ (None, 73, 73, 64)     │              0 │ re_lu_1346[0][0]       │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1347 (Conv2D)      │ (None, 73, 73, 80)     │          5,200 │ max_pooling2d_122[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1347  │ (None, 73, 73, 80)     │            240 │ conv2d_1347[0][0]      │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_1347 (ReLU)         │ (None, 73, 73, 80)     │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1348 (Conv2D)      │ (None, 71, 71, 192)    │        138,432 │ re_lu_1347[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1348  │ (None, 71, 71, 192)    │            576 │ conv2d_1348[0][0]      │
│ (BatchNormalization)      │                        │                │                        │
├──────────────────────

 Total params: 26,274,632 (100.23 MB)

 Trainable params: 26,241,864 (100.10 MB)

 Non-trainable params: 32,768 (128.00 KB)